In [1]:
import pandas as pd
import numpy as np

In [27]:
k = 0.8
Iplan = 100
ventas = [10,15,20,5,10,25,10,15,20,5] * 3
demora = [5,3,4,6,2,3,1,5,4,5] * 3

def calc_pedido(k, V, I, y, X):
    #if X[-1] > 60:
     #   return 0
    if len(X) == 3:
        weighted_avg = 0.1 * X[0] + 0.3 * X[1] + 0.5 * X[2] 
        if weighted_avg > 25:
            return 0   
        else:
            return V + k * (I - y)
    else:
        return V + k * (I - y)

def reposicion_ventas(Iplan, Y, X):
    reposicion = 0
    if Y < 80:
            reposicion = Iplan - Y
    elif (Y > 105):
        reposicion = 0
    else:
        reposicion = np.random.normal(loc=np.abs(Iplan-Y),scale=np.abs(Iplan-Y)/6)
    if reposicion > X:
        return X
    return reposicion
def costo_mantenimiento(X):
    return X*10
def delivery(spreadsheet, index):
    indexes = (np.arange(0,index).tolist()[::-1])
    count = 1
    acum = 0
    for idx in indexes:
        if(spreadsheet['Demora'].values[idx] == count):
            acum += spreadsheet['Pedido'].values[idx]
        count += 1
    return acum
keys = ['Tiempo','Ventas','Demora','Llegada','Pedido','Reposicion','Y','X','CP','CM','CT']

In [28]:
ss_dict = {
    'Tiempo':[],
    'Ventas':[],
    'Demora':[],
    'Llegada':[],
    'Pedido':[],
    'Reposicion':[],
    'Y':[],
    'X':[],
    'CP':[],
    'CM':[],
    'CT':[],    
}

In [33]:
spreadsheet = pd.DataFrame(ss_dict)
spreadsheet

,Tiempo,Ventas,Demora,Llegada,Pedido,Reposicion,Y,X,CP,CM,CT


In [31]:
def step(spreadsheet, index):
    temp = []
    #tiempo
    temp.append(index)
    if index == 0:
        #V
        temp.append(0)
        #D
        temp.append(0)
        #llegada
        temp.append(0)
        #pedido
        temp.append(0)
        #Reposicion
        temp.append(0)
        #Y
        temp.append(Iplan)
        #X
        X = np.mean(ventas) * np.mean(demora)
        temp.append(X)
        #CP
        temp.append(0)
        #CM
        temp.append(costo_mantenimiento(X))
        #CT
        temp.append(temp[-1] + temp[-2])
    else:
        #V
        temp.append(ventas[index-1])
        #D
        temp.append(demora[index-1])
        #llegada
        llega_stock = delivery(spreadsheet, index) 
        temp.append(llega_stock)
        #Y
        Y = spreadsheet['Y'].values[-1] - ventas[index-1]
        #pedido
        X = spreadsheet['X'].values[-1] + llega_stock

        P = calc_pedido(k, ventas[index], Iplan, Y, spreadsheet['X'].values[-3::])
        temp.append(P)
        #Reposicion

        R = reposicion_ventas(Iplan, Y, X)
        X = spreadsheet['X'].values[-1] - R + llega_stock
        temp.append(R)
        #Y asignacion
        temp.append(Y+R)
        #X
        temp.append(X)
        #CP
        if P !=0:
            CP = 1000 + 10*P
            temp.append(CP)
        else:
            temp.append(0)
        #CM
        temp.append(costo_mantenimiento(X))
        #CT
        temp.append(temp[-1] + temp[-2])

    df_temp = pd.DataFrame(dict(zip(keys, temp)), index=[0])
    return pd.concat([spreadsheet, df_temp], sort=False)

Revisar reposicion, ya que hay que acualizar el X cuando se repone, pero actualmente estamos actualizando el X en el index actual y actualizando Y en el index siguiente.

In [32]:
spreadsheet = pd.DataFrame(ss_dict)
for i in np.arange(0,30):
    spreadsheet = step(spreadsheet, i)
    
print('Costo total = ' + str(spreadsheet['CT'].sum()) )
spreadsheet


Costo total = 27684.5060021


,Tiempo,Ventas,Demora,Llegada,Pedido,Reposicion,Y,X,CP,CM,CT
0,0.0,0.0,0.0,0.000000,0.000000,0.000000,100.000000,51.300000,0.000000,513.000000,513.000000
0,1.0,10.0,5.0,0.000000,23.000000,9.439852,99.439852,41.860148,1230.000000,418.601484,1648.601484
0,2.0,15.0,3.0,0.000000,32.448119,13.262947,97.702799,28.597201,1324.481187,285.972011,1610.453198
0,3.0,20.0,4.0,0.000000,0.000000,22.297201,100.000000,6.300000,0.000000,63.000000,63.000000
0,4.0,5.0,6.0,0.000000,14.000000,5.688493,100.688493,0.611507,1140.000000,6.115065,1146.115065
0,5.0,10.0,2.0,32.448119,32.449205,12.653348,103.341841,20.406278,1324.492052,204.062777,1528.554829
0,6.0,25.0,3.0,23.000000,27.326527,21.658159,100.000000,21.748119,1273.265272,217.481187,1490.746459
0,7.0,10.0,1.0,32.449205,23.000000,9.674253,99.674253,44.523071,1230.000000,445.230711,1675.230711
0,8.0,15.0,5.0,23.000000,0.000000,14.582637,99.256890,52.940434,0.000000,529.404342,529.404342
0,9.0,20.0,4.0,27.326527,0.000000,20.743110,100.000000,59.523851,0.000000,595.238511,595.238511
